In [105]:
import numpy as np
import scipy.linalg as sln
import matplotlib.pyplot as plt
import warnings
import mpmath as mp
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

Points
<br># -> Doc(Main)
<br>## -> My Notes
<br>### -> Missing, Corrections to do

In [106]:
def mv(A, v, transp_flag):
    if transp_flag == 0:
        return A @ v
    else:
        return A.T @ v

In [107]:
def unv(j, n):
    e = np.zeros(n)
    e[j] = 1
    return e

In [108]:
###Get rid of function aspect
def element(A, i=None, j=None, n=None):
    # Handling default arguments
    if i is None:
        i = slice(None)  # Selecting all rows
    if j is None:
        j = slice(None)  # Selecting all columns

    if isinstance(A, np.ndarray):  # Matrix
        if min(A.shape) > 1:  # Check if A is not a vector
            if isinstance(i, list) and isinstance(j, list):
                e = A[np.ix_(i, j)]
            else:
                e = A[i, j]
        else:
            e = A[i] if isinstance(i, list) else A[i]
    else:  # Function
        if j is None:
            raise ValueError("j has to be nonempty when A is a function")
        e = mv(A, unv(j, n), 0)
        if i is not None:
            e = e[i]

    return e

In [109]:
def krylov_ata(A, v1=None, k=10, full=1, reortho=2):
    if v1 is None:
        v1 = np.random.randn(A.shape[1])

    if not np.issubdtype(A.dtype, np.number):
        raise ValueError("Matrix A should be numeric")

    alpha = np.zeros(k)
    beta = np.zeros(k if full else k-1)

    if reortho:
        V = np.zeros((len(v1), k + 1))
        V[:, 0] = v1 / np.linalg.norm(v1)
        U = np.zeros((A.shape[0], k))
    else:
        v = v1 / np.linalg.norm(v1)

    for j in range(k):
        if reortho:
            r = mv(A, V[:, j], 0)
            print(r)
            if j == 0 and reortho == 2:
                U = np.zeros((len(r), k))
        else:
            r = mv(A, v, 0)

        if j > 0:
            if reortho == 2:
                ###Fix indexing
                r -= beta[j-1] * U[:, j-1]
                r -= U[:, :j] @ (U[:, :j].T @ r)
            else:
                ###Fix indexing
                r -= beta[j-1] * u
        ###Fix indexing
        alpha[j] = np.linalg.norm(r)
        if alpha[j] == 0:
            break

        if reortho == 2:
            U[:, j] = r / alpha[j]
            r = mv(A, U[:, j], 1)
        else:
            u = r / alpha[j]
            r = mv(A, u, 1)

        if reortho:
            r -= alpha[j] * V[:, j]
            r -= V[:, :j+1] @ (V[:, :j+1].T @ r)
        else:
            r -= alpha[j] * v

        if j < k - 1 or full:
            beta[j] = np.linalg.norm(r)
            if beta[j] == 0:
                break

            if reortho:
                V[:, j+1] = r / beta[j]
            else:
                v = r / beta[j]

    if not reortho:
        V = v
    if reortho < 2:
        U = u

    
    return V, U, alpha, beta

In [110]:
def krylov_ata_expand(A, V, U, c, k=10):
    m = V.shape[1]
    V = np.concatenate((V, np.zeros((V.shape[0], k))), axis=1)
    U = np.concatenate((U, np.zeros((U.shape[0], k))), axis=1)
    alpha = np.zeros(k)
    beta = np.zeros(k)

    for j in range(m - 1, k + m):
        if j == m - 1:
            ###Check difference with imatlab
            r = mv(A, V[:, j-1], 0) - U[:, :j-1] @ c[:j-1]
        else:
            r = mv(A, V[:, j-1], 0) - beta[j-m-1] * U[:, j-2]

        r = r - U[:, :j-1] @ (U[:, :j-1].T @ r)
        alpha[j-m] = np.linalg.norm(r)
        if alpha[j-m] == 0:
            break
        U[:, j-1] = r / alpha[j-m]
        r = mv(A.T, U[:, j-1], 1) - alpha[j-m] * V[:, j-1]
        r = r - V[:, :j] @ (V[:, :j].T @ r)
        beta[j-m] = np.linalg.norm(r)
        if beta[j-m] == 0:
            break
        V[:, j] = r / beta[j-m]

    return V, U, alpha, beta

In [111]:
def krylov_schur_svd(A, **kwargs):
    nr = kwargs.get('nr', 1)
    v1 = kwargs.get('v1', None)
    tol = kwargs.get('tol', 1e-6)
    absrel = kwargs.get('absrel', 'rel')
    mindim = kwargs.get('mindim', 10)
    maxdim = kwargs.get('maxdim', 20)
    maxit = kwargs.get('maxit', 1000)
    target = kwargs.get('target', np.inf)
    info = kwargs.get('info', 0)

    if v1 is None:
        v1 = np.random.rand(A.shape[1])
    
    if mindim < nr:
        mindim = nr
    if maxdim < 2 * mindim:
        maxdim = 2 * mindim
    
    if absrel == 'rel' and np.issubdtype(A.dtype, np.number):
        tol = tol * np.linalg.norm(A, 1)

    B = np.zeros((maxdim, maxdim + 1))
    V, U, alpha, beta = krylov_ata(A, v1, mindim)
    B[:mindim + 1, :mindim + 1] = np.diag(np.append(alpha, [0])) + np.diag(beta, 1)
    hist = np.zeros(maxit)

    for k in range(maxit):
        V, U, alpha, beta = krylov_ata_expand(A, V, U, B[:mindim, mindim], maxdim - mindim)
        B[mindim: maxdim, mindim: maxdim] = np.diag(alpha) + np.diag(beta[:maxdim - mindim - 1], 1)
        B[maxdim - 1, maxdim] = beta[maxdim - mindim - 1]
        X, sigma, Y = np.linalg.svd(B[:maxdim, :maxdim])
        
        # Restart of Lanczos algorithm
        V = np.concatenate((element(V[:, :maxdim] @ Y, list(range(V.shape[0])), list(range(mindim))), V[:, maxdim:maxdim + 1]), axis=1)
        U = element(U[:, :maxdim] @ X, list(range(U.shape[0])), list(range(mindim)))
        c = B[:, maxdim]
        e = (c @ X)[:mindim]
        B[:mindim, :mindim + 1] = np.concatenate((np.diag(sigma[:mindim]), e.reshape(-1, 1)), axis=1)
        
        err = np.linalg.norm(e[:nr])
        hist[k] = err
        
        if info > 1:
            print(f'{k:4d}  {err:6.2e}')
            print(sigma[:min(3,nr)])
        
        if err < tol:
            sigma = sigma[:nr]
            V = V[:, :nr]
            U = U[:, :nr]
            mvs = np.arange(1, k + 1) * (maxdim - mindim) + mindim
            print(f"Found after {k} iterations with residual = {err:6.2e}")
            return sigma, V, U, hist[:k+1], mvs
    
    if info:
        print(f"Quit after max {k} iterations with residual = {err:6.2e}")
    sigma = sigma[:mindim]
    V = V[:, :mindim]
    return sigma, V, U, hist, mvs

Tests

In [112]:
# Fixed input matrices and vectors
A = np.array([
    [1, 2, 3, 4],
    [4, 5, 6, 7],
    [7, 8, 9, 10],
    [10, 11, 12, 13]
])
V = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])
U = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])
c = np.array([1, 2, 3])
k = 4

# Call the krylov_ata_expand function
V_exp, U_exp, alpha, beta = krylov_ata_expand(A, V, U, c, k)

# Display the results
print("V_exp:")
print(V_exp)
print("U_exp:")
print(U_exp)
print("alpha:")
print(alpha)
print("beta:")
print(beta)

V_exp:
[[ 1.0000  2.0000  0.1109  0.1110  0.1110  0.1110  0.1110]
 [ 4.0000  5.0000  0.3283  0.3283  0.3283  0.3283  0.3283]
 [ 7.0000  8.0000  0.5457  0.5457  0.5457  0.5457  0.5457]
 [ 10.0000  11.0000  0.7630  0.7630  0.7630  0.7630  0.7630]]
U_exp:
[[ 1.0000 -0.0769 -0.0776 -0.0776 -0.0776 -0.0776  0.0000]
 [ 4.0000 -0.3101 -0.3105 -0.3105 -0.3105 -0.3105  0.0000]
 [ 7.0000 -0.5432 -0.5433 -0.5433 -0.5433 -0.5433  0.0000]
 [ 10.0000 -0.7764 -0.7762 -0.7762 -0.7762 -0.7762  0.0000]]
alpha:
[ 62808872265.0779  3980876294657805.0000  254490739020159713280.0000
  16409014952123982162165760.0000]
beta:
[ 23837582699001.8320  1514825833565455616.0000
  97094763416731768586240.0000  6276870322158604508760899584.0000]


In [9]:
A = np.random.rand(50, 50)
sigma, V, U, hist, mvs = krylov_schur_svd(A, nr=3, tol=1e-4, maxit=100, info=2)

[ 3.3724  3.4100  3.0674  3.3790  3.0359  2.8383  3.2841  3.1066  2.5305
  3.7657  3.2284  3.2075  3.1913  3.0866  3.8066  3.4378  3.1540  3.1392
  3.0817  3.1131  3.0313  3.3383  2.8408  3.2810  3.0324  3.5173  2.9398
  2.9173  2.9508  3.4994  3.8583  3.2841  3.3910  2.8145  3.2498  3.4573
  3.2481  3.5145  2.7571  3.3285  2.8032  3.4978  3.2398  3.3497  3.5998
  3.0226  3.5455  3.2322  2.9413  3.2026]
[ 1.2540  1.7884  0.9667  1.7675  1.1029  1.5979  1.2612  1.5957  1.2095
  1.6654  1.9257  1.4173  1.6375  1.6379  1.4763  1.9490  1.2589  1.4336
  1.6166  1.4608  2.0765  1.5224  1.0175  1.6398  1.9746  1.9309  1.9753
  1.6223  1.5695  0.9209  1.4582  1.5403  1.3159  1.7744  1.5160  1.7260
  1.9761  1.5914  1.5442  1.5751  1.6004  1.8663  1.8188  1.6618  1.6040
  1.9044  1.6908  1.6011  1.7612  1.4502]
[-0.5522 -0.0707 -0.5027  0.3285 -0.6377  0.0489 -0.0127  0.4973 -0.2247
 -0.4833 -0.3106 -0.2292 -0.0237 -0.0238 -0.2521  0.2587 -0.4704 -0.2228
 -0.1131 -0.1951  0.7036 -0.5726 -0.1385

In [10]:
A = np.array([[1,2,3,4],[3,5,2,8],[5,3,45,6],[2,1,6,3]])
print(A)
V,U,alpha,beta = krylov_ata(A)
#print(V)
#print(U)
#print(alpha)
#print(beta)

[[ 1  2  3  4]
 [ 3  5  2  8]
 [ 5  3 45  6]
 [ 2  1  6  3]]
[ 0.3449  1.7199 -14.6981 -2.1105]
[-3.9755 -4.3037 -43.3405 -6.1832]
[-3.7243 -8.9701 -0.7513 -2.6862]
[-0.4544  0.2406  0.0626 -0.3144]
[-0.8944  1.7889 -37.5659 -4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[[-0.7485  0.1483 -0.5396  0.3559  0.0000 -0.0000 -0.0000 -0.0000 -0.0000
  -0.0000 -0.0000]
 [ 0.5346 -0.2439 -0.3179  0.7440  0.0000  0.0000  0.0000  0.0000  0.0000
   0.0000  0.0000]
 [-0.3110 -0.9357  0.1660 -0.0124 -0.8944  0.8944  0.8944  0.8944  0.8944
   0.8944  0.8944]
 [ 0.2393 -0.2072 -0.7617 -0.5653  0.4472 -0.4472 -0.4472 -0.4472 -0.4472
  -0.4472 -0.4472]]
[[ 0.0231 -0.4704  0.3069 -0.8270 -0.9701 -0.9701 -0.9701 -0.9701 -0.9701
  -0.9701]
 [ 0.1150 -0.8753 -0.1674  0.4389  0.2425  0.2425  0.2425  0.2425  0.2425
   0.2425]
 [-0.9830 -0.1117  0.1210  0.0810  0.0000 -0.000

In [10]:
A = np.array([[1,2,3,4],[3,5,2,8],[5,3,45,6],[2,1,6,3]])
print(A)
V,U,alpha,beta = krylov_ata(A)
#print(V)
#print(U)
#print(alpha)
#print(beta)

[[ 1  2  3  4]
 [ 3  5  2  8]
 [ 5  3 45  6]
 [ 2  1  6  3]]
[ 0.3449  1.7199 -14.6981 -2.1105]
[-3.9755 -4.3037 -43.3405 -6.1832]
[-3.7243 -8.9701 -0.7513 -2.6862]
[-0.4544  0.2406  0.0626 -0.3144]
[-0.8944  1.7889 -37.5659 -4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[ 0.8944 -1.7889  37.5659  4.0249]
[[-0.7485  0.1483 -0.5396  0.3559  0.0000 -0.0000 -0.0000 -0.0000 -0.0000
  -0.0000 -0.0000]
 [ 0.5346 -0.2439 -0.3179  0.7440  0.0000  0.0000  0.0000  0.0000  0.0000
   0.0000  0.0000]
 [-0.3110 -0.9357  0.1660 -0.0124 -0.8944  0.8944  0.8944  0.8944  0.8944
   0.8944  0.8944]
 [ 0.2393 -0.2072 -0.7617 -0.5653  0.4472 -0.4472 -0.4472 -0.4472 -0.4472
  -0.4472 -0.4472]]
[[ 0.0231 -0.4704  0.3069 -0.8270 -0.9701 -0.9701 -0.9701 -0.9701 -0.9701
  -0.9701]
 [ 0.1150 -0.8753 -0.1674  0.4389  0.2425  0.2425  0.2425  0.2425  0.2425
   0.2425]
 [-0.9830 -0.1117  0.1210  0.0810  0.0000 -0.000

In [11]:
def krylov_ata(A, v1=None, k=10, full=1, reortho=2):
    if v1 is None:
        v1 = np.random.randn(A.shape[1])

    if not np.issubdtype(A.dtype, np.number):
        raise ValueError("Matrix A should be numeric")

    alpha = np.zeros(k)
    beta = np.zeros(k if full else k-1)

    if reortho:
        V = np.zeros((len(v1), k + 1))
        V[:, 0] = v1 / np.linalg.norm(v1)
        U = np.zeros((A.shape[0], k))
    else:
        v = v1 / np.linalg.norm(v1)

    for j in range(k):
        if reortho:
            r = mv(A, V[:, j], 0)
            #print(r)
            if j == 0 and reortho == 2:
                U = np.zeros((len(r), k))
        else:
            r = mv(A, v, 0)

        if j > 0:
            if reortho == 2:
                r -= beta[j-1] * U[:, j-1]
                r -= U[:, :j] @ (U[:, :j].T @ r)
            else:
                r -= beta[j-1] * u
        alpha[j] = np.linalg.norm(r)
        if alpha[j] == 0:
            break

        if reortho == 2:
            U[:, j] = r / alpha[j]
            r = mv(A, U[:, j], 1)
        else:
            u = r / alpha[j]
        if reortho:
            r -= alpha[j] * V[:, j]
            r -= V[:, :j+1] @ (V[:, :j+1].T @ r)
        else:
            r -= alpha[j] * v

        if j < k - 1 or full:
            beta[j] = np.linalg.norm(r)
            if beta[j] == 0:
                break

            if reortho:
                V[:, j+1] = r / beta[j]
            else:
                v = r / beta[j]

        print(j)
        print(r)
        print(beta)
        print(alpha)

    if not reortho:
        V = v
    if reortho < 2:
        U = u

    
    return V, U, alpha, beta


In [12]:
def krylov_ata(A, v1=None, k=10, full=1, reortho=2):
    if v1 is None:
        v1 = np.random.randn(A.shape[1])

    if not np.issubdtype(A.dtype, np.number):
        raise ValueError("Matrix A should be numeric")

    alpha = np.zeros(k)
    beta = np.zeros(k if full else k-1)

    if reortho:
        V = np.zeros((len(v1), k + 1))
        V[:, 0] = v1 / np.linalg.norm(v1)
        U = np.zeros((A.shape[0], k))
    else:
        v = v1 / np.linalg.norm(v1)

    for j in range(k):
        if reortho:
            r = mv(A, V[:, j], 0)
            #print(r)
            if j == 0 and reortho == 2:
                U = np.zeros((len(r), k))
        else:
            r = mv(A, v, 0)

        if j > 0:
            if reortho == 2:
                r -= beta[j-1] * U[:, j-1]
                r -= U[:, :j] @ (U[:, :j].T @ r)
            else:
                r -= beta[j-1] * u
        alpha[j] = np.linalg.norm(r)
        if alpha[j] == 0:
            break

        if reortho == 2:
            U[:, j] = r / alpha[j]
            r = mv(A, U[:, j], 1)
        else:
            u = r / alpha[j]
        if reortho:
            r -= alpha[j] * V[:, j]
            r -= V[:, :j+1] @ (V[:, :j+1].T @ r)
        else:
            r -= alpha[j] * v

        if j < k - 1 or full:
            beta[j] = np.linalg.norm(r)
            if beta[j] == 0:
                break

            if reortho:
                V[:, j+1] = r / beta[j]
            else:
                v = r / beta[j]

        print(j)
        print(r)
        print(beta)
        print(alpha)

    if not reortho:
        V = v
    if reortho < 2:
        U = u

    
    return V, U, alpha, beta


In [113]:
A = np.array([[1,2,3,4],[3,5,2,8],[5,3,45,6],[2,1,6,3]])
v1 = np.array([1,4,3,7])
A = np.array([
    [1, 2, 3, 4],
    [3, 5, 2, 8],
    [5, 3, 45, 6],
    [2, 1, 6, 3]
], dtype=np.float64)
v1 = np.array([1, 4, 3, 7], dtype=np.float64)
v1

array([ 1.0000,  4.0000,  3.0000,  7.0000])

In [114]:
#Shows step by step changes to alpha and beta
V,U,alpha,beta = krylov_ata(A, v1, k = 4)

[ 5.3116  9.8150  22.4012  5.1962]
[ 1.2869 -1.2416  39.8975  4.6565]
[-0.2627 -2.0142  1.1686 -0.9649]
[-0.2483  0.2610  0.0886 -0.6213]


In [115]:
V

array([[ 0.1155,  0.0895, -0.9881, -0.0477,  0.2563],
       [ 0.4619, -0.1859, -0.0047,  0.8672,  0.6835],
       [ 0.3464,  0.9297,  0.1240,  0.0155,  0.6835],
       [ 0.8083, -0.3050,  0.0907, -0.4954,  0.0000]])

In [116]:
U

array([[ 0.2078, -0.3316,  0.6418, -0.6595],
       [ 0.3840, -0.8085, -0.0940,  0.4360],
       [ 0.8764,  0.4655,  0.0649,  0.1052],
       [ 0.2033, -0.1407, -0.7583, -0.6032]])

In [17]:
beta

array([ 35.7029,  2.3952,  0.2930,  0.0000,  34.4826,  206.8959,
        1551.7191,  19172.3513,  384309.0923,  11530514.1452])

In [18]:
alpha

array([ 25.5609,  18.4932,  0.8281,  0.6616,  0.0000,  68.9653,  482.7570,
        4758.6052,  76827.3358,  1921717.8749])

In [ ]:
1382600

In [59]:
#Version without MPF
def krylov_ata(A, v1=None, k=10, full=1, reortho=2):
    if v1 is None:
        v1 = np.random.randn(A.shape[1])

    # Ensure A and v1 are of type float64
    A = A.astype(np.float64)
    v1 = v1.astype(np.float64)

    if not np.issubdtype(A.dtype, np.number):
        raise ValueError("Matrix A should be numeric")

    alpha = np.zeros(k, dtype=np.float64)
    beta = np.zeros(k if full else k-1, dtype=np.float64)

    if reortho:
        V = np.zeros((len(v1), k + 1), dtype=np.float64)
        V[:, 0] = v1 / np.linalg.norm(v1)
        U = np.zeros((A.shape[0], k), dtype=np.float64)
    else:
        v = v1 / np.linalg.norm(v1)

    for j in range(k):
        if reortho:
            r = mv(A, V[:, j], 0)
            if j == 0 and reortho == 2:
                U = np.zeros((len(r), k), dtype=np.float64)
        else:
            r = mv(A, v, 0)
        print(f"Iteration {j+1}")    
        print(r)
        if j > 0:
            if reortho == 2:
                r -= beta[j-1] * U[:, j-1]
                r -= U[:, :j] @ (U[:, :j].T @ r)
            else:
                r -= beta[j-1] * u
        print(r)
        alpha[j] = np.linalg.norm(r)
        if alpha[j] == 0:
            break

        if reortho == 2:
            U[:, j] = r / alpha[j]
            r = mv(A, U[:, j], 1)
            print(r)
        else:
            u = r / alpha[j]

        if reortho:
            r -= alpha[j] * V[:, j]
            print(alpha[j] * V[:, j])
            print(r)
            r -= V[:, :j+1] @ (V[:, :j+1].T @ r)
            #print(r)
        else:
            r -= alpha[j] * v
            print(r)
        #print(r)
        if j < k - 1 or full:
            beta[j] = np.linalg.norm(r)
            if beta[j] == 0:
                break

            if reortho:
                V[:, j+1] = r / beta[j]
            else:
                v = r / beta[j]

        #print(f"Iteration {j+1}")
        #print(r)
        print()
        #print("Alpha:", alpha)
        #print("Beta:", beta)
        #print()

    if not reortho:
        V = v
    if reortho < 2:
        U = u

    return V, U, alpha, beta

# Test the function with the given inputs
A = np.array([
    [1, 2, 3, 4],
    [3, 5, 2, 8],
    [5, 3, 45, 6],
    [2, 1, 6, 3]
], dtype=np.float64)
v1 = np.array([1, 4, 3, 7], dtype=np.float64)

V, U, alpha, beta = krylov_ata(A, v1)

# Print results
print("V:", V)
print("U:", U)
print("Alpha:", alpha)
print("Beta:", beta)


Iteration 1
[ 5.3116  9.8150  22.4012  5.1962]
[ 5.3116  9.8150  22.4012  5.1962]
[ 6.1482  5.1680  42.0484  9.7712]
[ 2.9515  11.8061  8.8546  20.6606]
[ 3.1967 -6.6381  33.1938 -10.8894]

Iteration 2
[ 1.2869 -1.2416  39.8975  4.6565]
[-6.1323 -14.9509  8.6081 -2.6014]
[-0.7109 -3.4497  17.4906 -5.4232]
[ 1.6558 -3.4384  17.1936 -5.6405]
[-2.3667 -0.0113  0.2970  0.2173]

Iteration 3
[-0.2627 -2.0142  1.1686 -0.9649]
[ 0.5315 -0.0778  0.0537 -0.6280]
[-0.8323  0.2502  0.1072 -0.0700]
[-0.8183 -0.0039  0.1027  0.0751]
[-0.0140  0.2541  0.0045 -0.1452]

Iteration 4
[-0.2483  0.2610  0.0886 -0.6213]
[-0.4363  0.2885  0.0696 -0.3991]
[-0.0315  0.5738  0.0103 -0.3278]
[-0.0315  0.5738  0.0103 -0.3278]
[ 0.0000  0.0000 -0.0000  0.0000]

Iteration 5
[ 3.6737  5.5533  34.0889  5.2970]
[ 0.0000  0.0000 -0.0000 -0.0000]
[-5.2095 -3.1009 -45.3967 -6.3258]
[ 0.0000  0.0000  0.0000  0.0000]
[-5.2095 -3.1009 -45.3967 -6.3258]

Iteration 6
[ 3.6737  5.5533  34.0889  5.2970]
[-0.0000  0.0000 -68.432

In [100]:
import numpy as np
import mpmath
from mpmath import mp

# Set the precision to 50 decimal places (adjust as needed)
##precison 50 gives a smaller difference
mp.dps = 14

def mv(A, v, trans):
    if trans == 0:
        return A @ v
    else:
        return A.T @ v

def krylov_ata(A, v1=None, k=10, full=1, reortho=2):
    if v1 is None:
        v1 = np.random.randn(A.shape[1])

    # Ensure A and v1 are of type mpf
    A = np.array(A, dtype=object)
    v1 = np.array(v1, dtype=object)

    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            A[i, j] = mp.mpf(A[i, j])

    for i in range(v1.shape[0]):
        v1[i] = mp.mpf(v1[i])

    alpha = np.zeros(k, dtype=object)
    beta = np.zeros(k if full else k-1, dtype=object)

    if reortho:
        V = np.zeros((len(v1), k + 1), dtype=object)
        V[:, 0] = v1 / mp.norm(v1)
        U = np.zeros((A.shape[0], k), dtype=object)
    else:
        v = v1 / mp.norm(v1)

    for j in range(k):
        if reortho:
            r = mv(A, V[:, j], 0)
            if j == 0 and reortho == 2:
                U = np.zeros((len(r), k), dtype=object)
        else:
            r = mv(A, v, 0)
        #print(f"Iteration {j+1}")    
        #print([mp.nstr(val, 4) for val in r])
        if j > 0:
            if reortho == 2:
                r -= beta[j-1] * U[:, j-1]
                r -= U[:, :j] @ (U[:, :j].T @ r)
            else:
                r -= beta[j-1] * u
        #print([mp.nstr(val, 4) for val in r])
        alpha[j] = mp.norm(r)
        if alpha[j] == 0:
            break

        if reortho == 2:
            U[:, j] = r / alpha[j]
            r = mv(A, U[:, j], 1)
            #print([mp.nstr(val, 4) for val in r])
        else:
            u = r / alpha[j]

        if reortho:
            r -= alpha[j] * V[:, j]
            #print([mp.nstr(-val, 4) for val in (alpha[j] * V[:, j])])
            #print([mp.nstr(val, 4) for val in r])
            r -= V[:, :j+1] @ (V[:, :j+1].T @ r)
            #print([mp.nstr(val, 4) for val in r])
        else:
            r -= alpha[j] * v
            #print([mp.nstr(val, 4) for val in r])
        if j < k - 1 or full:
            beta[j] = mp.norm(r)
            if beta[j] == 0:
                break

            if reortho:
                V[:, j+1] = r / beta[j]
            else:
                v = r / beta[j]

        #print()

    if not reortho:
        V = v
    if reortho < 2:
        U = u

    return V, U, alpha, beta

# Test the function with the given inputs
A = np.array([
    [1, 2, 3, 4],
    [3, 5, 2, 8],
    [5, 3, 45, 6],
    [2, 1, 6, 3]
], dtype=object)
v1 = np.array([1, 4, 3, 7], dtype=object)

V, U, alpha, beta = krylov_ata(A, v1)

# Print results
#print("V:", [[mp.nstr(val, 4) for val in row] for row in V])
#print("U:", [[mp.nstr(val, 4) for val in row] for row in U])
#print("Alpha:", [mp.nstr(val, 4) for val in alpha])
#print("Beta:", [mp.nstr(val, 4) for val in beta])
#

In [101]:
print(np.array([[mp.nstr(val, 4) for val in row] for row in U]))

[['0.2078' '-0.3316' '0.6418' '-0.6595' '-0.6667' '-0.6667' '-0.6667'
  '-0.6667' '-0.6667' '-0.6667']
 ['0.384' '-0.8085' '-0.09395' '0.436' '-0.6667' '-0.6667' '-0.6667'
  '-0.6667' '-0.6667' '-0.6667']
 ['0.8764' '0.4655' '0.06488' '0.1052' '0.0' '-4.432e-16' '-1.266e-16'
  '-3.725e-16' '-4.352e-16' '-4.256e-16']
 ['0.2033' '-0.1407' '-0.7583' '-0.6032' '-0.3333' '-0.3333' '-0.3333'
  '-0.3333' '-0.3333' '-0.3333']]


In [102]:
print(np.array([[mp.nstr(val, 4) for val in row] for row in V]))

[['0.1155' '0.08954' '-0.9881' '-0.04768' '0.1562' '0.1562' '0.1562'
  '0.1562' '0.1562' '0.1562' '0.1562']
 ['0.4619' '-0.1859' '-0.00472' '0.8672' '0.937' '0.937' '0.937' '0.937'
  '0.937' '0.937' '0.937']
 ['0.3464' '0.9297' '0.124' '0.0155' '0.0' '8.863e-16' '2.954e-16'
  '2.954e-16' '1.403e-16' '2.969e-16' '3.166e-16']
 ['0.8083' '-0.305' '0.09072' '-0.4954' '0.3123' '0.3123' '0.3123'
  '0.3123' '0.3123' '0.3123' '0.3123']]


In [103]:
print("Alpha:", [mp.nstr(val, 4) for val in alpha])
print("Beta:", [mp.nstr(val, 4) for val in beta])

Alpha: ['25.56', '18.49', '0.8281', '0.6616', '1.066e-14', '16.03', '112.2', '1106.0', '1.786e+4', '4.468e+5']
Beta: ['35.7', '2.395', '0.293', '2.02e-29', '8.017', '48.1', '360.8', '4457.0', '8.935e+4', '2.681e+6']


In [104]:
A = np.array([
    [1, 2, 3, 4],
    [3, 5, 2, 8],
    [5, 3, 45, 6],
    [2, 1, 6, 3]
], dtype=object)
v1 = np.array([1, 4, 3, 7], dtype=object)

V, U, alpha, beta = krylov_ata(A, v1, k = 4)

# Print results
print("V:", [[mp.nstr(val, 4) for val in row] for row in V])
print("U:", [[mp.nstr(val, 4) for val in row] for row in U])
print("Alpha:", [mp.nstr(val, 4) for val in alpha])
print("Beta:", [mp.nstr(val, 4) for val in beta])


V: [['0.1155', '0.08954', '-0.9881', '-0.04768', '0.1562'], ['0.4619', '-0.1859', '-0.00472', '0.8672', '0.937'], ['0.3464', '0.9297', '0.124', '0.0155', '0.0'], ['0.8083', '-0.305', '0.09072', '-0.4954', '0.3123']]
U: [['0.2078', '-0.3316', '0.6418', '-0.6595'], ['0.384', '-0.8085', '-0.09395', '0.436'], ['0.8764', '0.4655', '0.06488', '0.1052'], ['0.2033', '-0.1407', '-0.7583', '-0.6032']]
Alpha: ['25.56', '18.49', '0.8281', '0.6616']
Beta: ['35.7', '2.395', '0.293', '2.02e-29']
